In [1]:
import cv2 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000

In [2]:
imgobj = cv2.imread('img/green_face.jpg',)
gray_img = cv2.cvtColor(imgobj, cv2.COLOR_BGR2GRAY)
gray_img = np.float32(gray_img)
corners = cv2.goodFeaturesToTrack(gray_img, maxCorners=50,     qualityLevel=0.20, minDistance=20)
corners = np.float32(corners)

for item in corners:
    x, y = item[0]

    cv2.circle(imgobj, (int(x), int(y)), 50, (255, 0, 0), -1)


cv2.imshow('',imgobj)
 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
def angle_cos(p0, p1, p2):
    import numpy as np

    d1, d2 = (p0-p1).astype('float'), (p2-p1).astype('float')
    return abs( np.dot(d1, d2) / np.sqrt( np.dot(d1, d1)*np.dot(d2, d2) ) )

In [ ]:
imgobj = cv2.imread('img/green_face.jpg',)
gray_img = cv2.cvtColor(imgobj, cv2.COLOR_BGR2GRAY)
gray_img = np.float32(gray_img)
blurred = cv2.GaussianBlur(gray_img, (3, 3), 0)
canny = cv2.Canny(gray_img, 40, 80)
cv2.imshow('',canny)
 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
# import the opencv library
import cv2
vid = cv2.VideoCapture(0) 
is_acquis=False;
while(True):   
    # Capture the video frame
    # by frame
    ret, frame = vid.read()


    font = cv2.FONT_HERSHEY_SIMPLEX
    gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_img = np.float32(gray_img)
    blurred = cv2.GaussianBlur(gray_img, (3, 3), 0)
    canny = cv2.Canny(frame, 20, 40)
    kernel = np.ones((3,3), np.uint8)
    dilated = cv2.dilate(canny, kernel, iterations=2)
    (contours, hierarchy) = cv2.findContours(dilated.copy(), cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    squares=[]
    for cnt in contours:
                    cnt_len = cv2.arcLength(cnt, True)
                    cnt = cv2.approxPolyDP(cnt, 0.04*cnt_len, True)
                    if len(cnt) == 4  and cv2.isContourConvex(cnt):
                        cnt = cnt.reshape(-1, 2)
                        max_cos = np.max([angle_cos( cnt[i], cnt[(i+1) % 4], cnt[(i+2) % 4] ) for i in range(4)])
                        if max_cos < 0.2 and cv2.contourArea(cnt)>5000 and  cv2.contourArea(cnt)<10000:
                            squares.append(cnt)

    cv2.drawContours(frame, squares, -1, (0,255,0), 3)

    if len(squares)==9:
            front_face=frame
            front_face_contours=squares
            color_of_cubes=distance_colors(get_contours(front_face,front_face_contours),dic_colors)
            is_acquis=True
    
    if is_acquis:
        cv2.putText(frame, 'Acquis !,tournez la face', (50, 50), font, 1, (0, 255, 255), 2, cv2.LINE_4)
        frame=draw_face((1000,590),(1010,600),20,frame,dic_colors,color_of_cubes)
            
    
    cv2.imshow('frame',frame)
    

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# After the loop release the cap object
vid.release()

cv2.destroyAllWindows()

In [4]:
def draw_face(pt1,pt2,of,frame,dic_colors,color_of_cubes):

    tab=[(-of,-of),(-of,0),(-of,of),(0,-of),(0,0),(0,of),(of,-of),(of,0),(of,of)]
    for i,x in enumerate(tab):
        cv2.rectangle(frame, (pt1[0]-x[0],pt1[1]-x[1]), (pt2[0]-x[0],pt2[1]-x[1]), tuple(dic_colors[color_of_cubes[i]]), thickness=10)
    return frame   

In [5]:
def get_contours(img,contours):
    tab=[]
    for cnt in contours:
        if cv2.contourArea(cnt) >800: # filter small contours
            x,y,w,h = cv2.boundingRect(cnt) # offsets - with this you get 'mask'
            tab.append(np.array(cv2.mean(img[y:y+h,x:x+w])).astype(np.uint8))
    return tab

In [6]:

def distance_colors(contours_colors,dic_colors):
    reco_colors=[]
    for contour in contours_colors:
        dist_min=np.inf
        for key,value in dic_colors.items():
            
            color_1 = convert_color(convert_color(sRGBColor(contour[2]/255,contour[1]/255, contour[0]/255), LabColor), LabColor);
            color_2 = convert_color(convert_color(sRGBColor(value[0]/255,value[1]/255,value[2]/255), LabColor), LabColor);
            delta_e = delta_e_cie2000(color_1, color_2)
    
            if delta_e<dist_min:
                dist_min=delta_e
                final_color=key
        reco_colors.append(final_color)
    return reco_colors

In [7]:
dic_colors= {"blue" :[0,88,144],"green":[14,154,57],"orange":[210,111,38],"red":[83,48,5],"yellow":[149,212,85],"white":[133,191,177]}

In [ ]:
color_of_cubes=distance_colors(get_contours(front_face,front_face_contours),dic_colors)
frame=draw_face((1000,590),(1010,600),20,frame,dic_colors,color_of_cubes)
plt.show